# RAG 구축하기
## 순서
1. 데이터 읽기(`document_loaders`)
2. 문장 분할(`Splitter`)
3. 임베딩(`Embeddings`)
4. 저장(`VectorStore`)
---
5. 검색기(`Retriever`)
6. 검색 성능 개선(`Ensemble Retriever`)
   

### 1. 데이터 읽기

In [1]:
import pandas as pd
df = pd.read_csv("output.csv")

temp_df = df['processed_text'].apply(lambda x: x.split("\n"))

postprocessed_text_list = []

for sentences in temp_df:
    for sentence in sentences:
        temp = sentence[3:].strip()
        postprocessed_text_list.append(temp)

postprocessed_text_list
text_data = "\n\n".join(postprocessed_text_list)


In [2]:
text = "장애인에 대한 시각과 태도로서 따로 존재하는 것이 아닙니다."

In [3]:
postprocessed_text_list.index(text)

120

In [4]:
postprocessed_text_list[120]

'장애인에 대한 시각과 태도로서 따로 존재하는 것이 아닙니다.'

### 1.1 TextLoader 사용

In [5]:

file = open('data.txt', 'w', encoding='utf-8')    
file.write(text_data)
file.close()

In [12]:
data

[Document(page_content='편견 을 걷어내면 인재 가 보입니다\n\n직장 내 장애인 인식개선 교육\n\n장애인 고용\n\n장애인 고용, 기업에게 어떤 의미인가? 함께 일하는 동료 이해하기\n\n장애인 고용, 이렇게 할 수 있습니다\n\n함께 일하기, 이렇게 하면 잘 할 수 있습니다\n\n목차: 01, 02, 03, 04\n\n장애인 고용, 기업에게 어떤 의미인가요?\n\n차별도 우대도 없었던 게임 산업 분위기 속 장애인 고용의 돌파구 2011 년 ㈜넥슨커뮤니케이션즈(넥슨컴즈)를 설립하고 넥슨의 게임 운영 서비스 지원과 고객 지원 업무를 담당함 * 게임 홈페이지 관리, 게임 상담, 모니터링 등 4개월간의 직무 훈련을 거친 장애인 23명이 처음으로 산업에 입사하게 됨.\n\n실력으로 당당히 No.1 게임 기업의 일원이 되다! 장애인을 고용하는 기업 사례: 넥슨컴즈의 장애인 고용 주력부서 - 웹서비스 운영팀. 상시 근로자 2,691명 / 장애인 근로자 33명(2.04%) * 2018.12월 기준 모든 출입문 문턱 제거, 통로 간격 확장, 핸드레일 78%가 장애인 직원을 위한 것, 이중 중증장애인 비율은 73%.\n\n장애인 업무의 한계를 없애다\n\n상시 근로자 100,016 명 / 장애 근로자 1,386 명 (1.54%) * 2018.12 월 기준\n\n장애물 없는 생활환경(Barrier Free) 자체 인증 제도를 각 사업장 도입\n\n직무갈등, 대인관계 등 고충을 상담심리사가 상주하며 해결\n\n업무에 집중할 환경을 마련\n\n장애인 고객의 입장에서 바라본 기술개발에 적극 참여\n\n장애인 = 단순한 업무? ‘삼성전자 맞춤형 인재’로 성장하는 데 있어 비장애인과 장애인의 구분은 의미가 없음 판단\n\n직무교육을 거쳐 고용된 장애인, 전기·전자·정보기술·기계디자인 분야 각 사업장 근무 중\n\n장애인고용 기업 사례: 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련\n\n기업 ESG 경영 전략으로의 \'장애인 고용\'\n\n환경, 사회, 

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data.txt", encoding='utf-8')
data = loader.load()

## 1.2 모델 불러오기

In [7]:
from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain import LLMChain

"""
<hyper parameter explanation>
- temperature : control of randomness(probability)
- top_p : cdf(누적확률분포) 컷팅 비율 (상위 몇프로 사용할 지)
- top_k : 답변으로 뽑아낼 샘플 개수
"""

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    temperature=0,
    top_p=0,
    top_k=1,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 분할(Splitter)

`chunk size`와 `overlap`은 문서마다 최적의 값이 문서마다 다를 수 있으니 최적의 값은 테스트 필요  
`RecursiveJsonSplitter`, `CharacterTextSplitter`, `RecursiveCharacterTextSplitter` 등이 있음  

### RecursiveCharacterTextSplitter
- "\n\n", "\n", " ", "" 재귀적으로 청킹

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    )

splits = text_splitter.split_documents(data)
print(f'len(splits[0].page_content) : {len(splits[0].page_content)}')
splits[0].page_content

len(splits[0].page_content) : 176


'편견 을 걷어내면 인재 가 보입니다\n\n직장 내 장애인 인식개선 교육\n\n장애인 고용\n\n장애인 고용, 기업에게 어떤 의미인가? 함께 일하는 동료 이해하기\n\n장애인 고용, 이렇게 할 수 있습니다\n\n함께 일하기, 이렇게 하면 잘 할 수 있습니다\n\n목차: 01, 02, 03, 04\n\n장애인 고용, 기업에게 어떤 의미인가요?'

In [57]:
print(type(splits))

<class 'list'>


## 3. 임베딩(Embedding)
텍스트를 벡터화 시켜 의미별 유사도 계산을 할 수 있게 해줌

In [9]:
"""
pip install -U langchain-huggingface
"""
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': device}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

c:\Users\dm705\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\dm705\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
splits_docs = list(map(lambda x:x.page_content, splits))
embedding = embedding_model.embed_documents(splits_docs)


## 4. 저장(`VectorStore`)

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

vectorstore = FAISS.from_documents(splits,
                                   embedding = embedding_model,
                                  )

# 로컬에 DB 저장
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore.save_local(MY_FAISS_INDEX)

## 5. 검색(`Retriever`)

In [67]:
# 로컬 DB 불러오기
vectorstore = FAISS.load_local(MY_FAISS_INDEX, 
                               embedding_model, 
                               allow_dangerous_deserialization=True # 잠재적으로 위험한 데이터 구조나 객체를 포함할 수 있는 인덱스 파일의 로딩을 허용. 주로 자신이 직접 생성하고 저장한 인덱스 파일을 로드할 때 사용
                               )

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출
retrieved_docs = retriever.invoke("장애인을 위한 정책")
retrieved_docs

[Document(page_content='정당한 편의를 위한 물적인 지원으로는 직무 재배치, 서비스 제도 개선, 시설 및 장비 설치나 개조, 출입구 및 경사로 설치 등이 있습니다 (장애인차별금지법 제4조 2항).\n\n직무 조정과 수어 통역 및 훈련 보조 인력 제공, 휴가제도나 사업장 방침 변경도 정당한 편의에 포함됩니다 (장애인차별금지법 제4조 2항).\n\n정당한 편의제공으로 장애인 동등한 활동 참여\n\n정당한 편의제공 거부에 의한 차별 지체장애 시각장애 청각장애 신장장애 의사소통 제한\n\n직무 수행을 위한 수어통역사 보조인 배치\n\n근무시간 중 투석이 필요한 경우 근무시간 변경하여 가능하도록 함\n\n모니터에 확대 독서기 설치\n\n계단 이용이 어렵다면 경사로로 출입 가능하도록 함\n\n장애 유형별 정당한 편의제공\n\n정당한 편의제공으로 장애인 동등한 활동 참여\n\n정당한 편의제공 거부에 의한 차별\n\n실내 근무 지체장애인 근로자\n\n외부업무 많은 지체장애인 근로자\n\n제조업체 생산직에 근무하는 지적장애인 근로자\n\n외식업체 주방보조로 근무하는 지적장애인 근로자\n\n생산성 향상을 위한 맞춤 보조공학기기 필요\n\n원활한 업무수행과 안전 위해 작업지도원 필요\n\n이동 및 짐 운반 도와줄 보조인 필요\n\n경사로, 엘리베이터, 높낮이 조절 테이블 등의 시설·장비가 필요함\n\n수행직무에 따른 정당한 편의제공\n\n일하고 싶다면 지원받을 수 있습니다.\n\n장애인 지원제도: 취업알선지원, 중증장애인 지원고용, 취업성공패키지.\n\n중증장애인 인턴제: 직무능력 향상과 정규직 전환 지원.\n\n선훈련 후배치 선배치 후훈련.\n\n장애인 사업주 훈련준비금, 일비, 상해보험 등 제공.\n\n일하고 싶으시다면 지원받을 수 있습니다.\n\n장애인 지원제도는 장애인이 일자리를 찾고, 유지할 수 있도록 도와줍니다.\n\n이 프로그램은 취업 지원뿐만 아니라 직업 능력 향상과 취업 기회를 높이기 위한 다양한 서비스를 제공합니다.\n\n상담 및 직업 계획 수

## 6. 검색성능 개선(`Ensemble`)
- 형태소 분석기 + BM25, FAISS 리트리버로 한글 검색 성능 개선

In [2]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [3]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

In [73]:
for split_text in splits:
    print(" ".join([token.form for token in kiwi.tokenize(split_text.page_content)]))

편견 을 걷 어 내 면 인재 가 보이 ᆸ니다 직장 내 장애인 인식 개선 교육 장애인 고용 장애인 고용 , 기업 에게 어떤 의미 이 ᆫ가 ? 함께 일 하 는 동료 이해 하기 장애인 고용 , 이렇 게 하 ᆯ 수 있 습니다 함께 일 하 기 , 이렇 게 하 면 잘 하 ᆯ 수 있 습니다 목차 : 01 , 02 , 03 , 04 장애인 고용 , 기업 에게 어떤 의미 이 ᆫ가 요 ? 차별 도 우대 도 없 었 던 게임 산업 분위기 속 장애인 고용 의 돌파구 2011 년 ㈜ 넥슨 커뮤니케이션 즈 ( 넥슨컴즈 ) 를 설립 하 고 넥슨 의 게임 운영 서비스 지원 과 고객 지원 업무 를 담당 하 ᆷ * 게임 홈페이지 관리 , 게임 상담 , 모니터링 등 4 개월 간 의 직무 훈련 을 거치 ᆫ 장애인 23 명 이 처음 으로 산업 에 입사 하 게 되 ᆷ . 실력 으로 당당히 No. 1 게임 기업 의 일원 이 되 다 ! 장애인 을 고용 하 는 기업 사례 : 넥슨 컴 즈 의 장애인 고용 주력 부서 - 웹 서비스 운영 팀 . 상시 근로자 2,691 명 / 장애인 근로자 33 명 ( 2.04 % ) * 2018.12 월 기준 모든 출입문 문턱 제거 , 통로 간격 확장 , 핸드 레일 78 % 가 장애인 직원 을 위하 ᆫ 것 , 이중 중증 장애인 비율 은 73 % . 장애인 업무 의 한계 를 없애 어 다 상시 근로자 100,016 명 / 장애 근로자 1,386 명 ( 1.54 % ) * 2018.12 월 기준 장애물 없 는 생활 환경 ( Barrier Free ) 자체 인증 제도 를 각 사업장 도입 직무 갈등 , 대인 관계 등 고충 을 상담 심리 사 가 상주 하 며 해결 업무 에 집중 하 ᆯ 환경 을 마련 장애 이 ᆫ 고객 의 입장 에서 바라보 ᆫ 기술 개발 에 적극 참여 장애인 = 단순 하 ᆫ 업무 ? ‘ 삼성전자 맞춤 형 인재 ’ 로 성장 하 는 데 있 어 비 장애인 과 장애인 의 구분 은 의미 가 없 음 판단 직무 교육 을 거치 어 고용 되 ᆫ 장애인 , 전기 · 전자 · 정보 기술 · 기계

In [74]:
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [34]:
bm25 = BM25Retriever.from_documents(splits)
kiwi_bm25 = BM25Retriever.from_documents(splits, preprocess_func=kiwi_tokenize)
faiss = FAISS.from_documents(splits, embedding_model).as_retriever()
bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)

kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

NameError: name 'dos' is not defined

In [ ]:
retrievers = {
    "bm25_faiss_73": bm25_faiss_73,
    "kiwibm25_faiss_37" : kiwibm25_faiss_37
}

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출

In [83]:
def print_search_results(retrivers, query):
    print(f"Query: {query}")
    
    for name, retriever in retrievers.items():
        print(f"{name}:  \t : {retriever.invoke(query)[0].page_content}")
        
    print("==="* 20)
    

In [86]:
print_search_results(retrievers, "장애인은 선천적으로 타고나는 건가요?")[0]

Query: 장애인은 선천적으로 타고나는 건가요?
bm25_faiss_73:  	 : 근로자 수 4 명 이하의 장애인 사업주 및 근로자 지원

보조공학기기: 출퇴근 보조, 시각장애인용 컴퓨터 음성증폭기, 맞춤보조 공학기기, 자동차 개조, 차량용 보조공학기기, 이동 지원, 수기 서류 대독, 수어 통역

"함께 일하기"

"이렇게 하면 잘할 수 있습니다"

장애인은 인간의 존엄과 가치를 가지며 행복을 추구할 권리를 가진 사람들입니다.

그들은 건전하고 책임 있는 사회 구성원으로 자신의 능력을 개발하여 자립하도록 노력해야 합니다.

국가와 사회는 헌법과 국제연합의 장애인 권리선언 정신에 따라 장애인 인권을 보호하고 완전한 사회 참여 및 평등, 그리고 더불어 살아가는 사회를 위한 여건과 환경을 조성해야 합니다.

인권과 장애인 인권: 누구나 예외 없이 보장되어야 하는 인권.

대한민국 헌법 제10조: 모든 국민은 인간으로서 존엄과 가치를 가지며 행복을 추구할 권리를 가집니다.

장애는 점진적으로 변화하는 개념이며, 손상을 지닌 사람과 그들이 다른 사람들과 동등하게 사회에 완전히 참여하고 효과적으로 참여하는 것을 저해하는 태도 및 환경적인 장벽 간의 상호작용으로부터 기인한다는 것을 인정합니다.

보호받는 신분, 편견, 차별의 대상이 되는 수동적, 의존적이며 부정적 시민 소외 무력함 박애, 자선, 시혜의 대상 _ UN 장애인 권리협약 전문 (e) 항 출처: 김형식 (2008) UN 장애인권리협약과 장애인 시민권 누구나 예외 없이 보장되어야 하는 인권 인권과 장애인 인권 UN 장애 권리협약 적극적 사회성원 평등 및 차별 금지 긍정적 시민적 권리 포함 역량 강화 시민으로서의 권리 인정 장애인도 시민으로서 인권을 누릴 권리가 있습니다.

장애인 인권에 대한 목표: 모든 인간의 보편적 권리로서의 장애인 인권 보장.

장애인과 비장애인 모두에게 공감대 확대가 중요하다.

장애인의 접근권한: 사회적 약자의 인권과 모든 사람의 인권은 무엇인가?
kiwibm25_faiss_37:  	

'근로자 수 4 명 이하의 장애인 사업주 및 근로자 지원\n\n보조공학기기: 출퇴근 보조, 시각장애인용 컴퓨터 음성증폭기, 맞춤보조 공학기기, 자동차 개조, 차량용 보조공학기기, 이동 지원, 수기 서류 대독, 수어 통역\n\n"함께 일하기"\n\n"이렇게 하면 잘할 수 있습니다"\n\n장애인은 인간의 존엄과 가치를 가지며 행복을 추구할 권리를 가진 사람들입니다.\n\n그들은 건전하고 책임 있는 사회 구성원으로 자신의 능력을 개발하여 자립하도록 노력해야 합니다.\n\n국가와 사회는 헌법과 국제연합의 장애인 권리선언 정신에 따라 장애인 인권을 보호하고 완전한 사회 참여 및 평등, 그리고 더불어 살아가는 사회를 위한 여건과 환경을 조성해야 합니다.\n\n인권과 장애인 인권: 누구나 예외 없이 보장되어야 하는 인권.\n\n대한민국 헌법 제10조: 모든 국민은 인간으로서 존엄과 가치를 가지며 행복을 추구할 권리를 가집니다.\n\n장애는 점진적으로 변화하는 개념이며, 손상을 지닌 사람과 그들이 다른 사람들과 동등하게 사회에 완전히 참여하고 효과적으로 참여하는 것을 저해하는 태도 및 환경적인 장벽 간의 상호작용으로부터 기인한다는 것을 인정합니다.\n\n보호받는 신분, 편견, 차별의 대상이 되는 수동적, 의존적이며 부정적 시민 소외 무력함 박애, 자선, 시혜의 대상 _ UN 장애인 권리협약 전문 (e) 항 출처: 김형식 (2008) UN 장애인권리협약과 장애인 시민권 누구나 예외 없이 보장되어야 하는 인권 인권과 장애인 인권 UN 장애 권리협약 적극적 사회성원 평등 및 차별 금지 긍정적 시민적 권리 포함 역량 강화 시민으로서의 권리 인정 장애인도 시민으로서 인권을 누릴 권리가 있습니다.\n\n장애인 인권에 대한 목표: 모든 인간의 보편적 권리로서의 장애인 인권 보장.\n\n장애인과 비장애인 모두에게 공감대 확대가 중요하다.\n\n장애인의 접근권한: 사회적 약자의 인권과 모든 사람의 인권은 무엇인가?'

In [14]:
from langchain_core.documents import Document

In [18]:
docs = [
    Document(
        page_content="장애 유형은 15가지로 분류됩니다. 외부기능 장애 = [지체 장애, 청각 장애, 뇌병변 장애] 등"
    ),
    Document(
        page_content="장애인은 선천적인 경우보다 후천적인 경우가 훨씬 많습니다."
    ),
    Document(
        page_content="장애인 업무의 한계를 없애다"
    ),
    Document(
        page_content="장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련."
    ),
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=10,
    )

docs_split = text_splitter.split_documents(docs)

embedding = embedding_model.embed_documents(list(map(lambda x:x.page_content, docs_split)))


from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

# 로컬에 DB 저장
vectorstore = FAISS.from_documents(docs,
                                   embedding = embedding_model,
                                  )

# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출

MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore.save_local(MY_FAISS_INDEX)


In [19]:
docs_split

[Document(page_content='장애 유형은 15가지로 분류됩니다. 외부기능 장애 ='),
 Document(page_content='외부기능 장애 = [지체 장애, 청각 장애, 뇌병변'),
 Document(page_content='장애, 뇌병변 장애] 등'),
 Document(page_content='장애인은 선천적인 경우보다 후천적인 경우가 훨씬'),
 Document(page_content='경우가 훨씬 많습니다.'),
 Document(page_content='장애인 업무의 한계를 없애다'),
 Document(page_content='장애인 고용 전문가와 함께 다양한 장애인 고용 대책'),
 Document(page_content='장애인 고용 대책 마련.')]

In [20]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]


In [39]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

bm25 = BM25Retriever.from_documents(docs_split)
kiwi_bm25 = BM25Retriever.from_documents(docs_split, preprocess_func=kiwi_tokenize)
faiss = FAISS.from_documents(docs_split, embedding_model).as_retriever(search_type="similarity", search_kwargs={"k": 5})
bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)

kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [48]:
def print_search_results(retrivers, query):
    print(f"Query: {query}")
    result = []

    for name, retriever in retrievers.items():
        print(f"{name}:  \t : {retriever.invoke(query)[0].page_content}")
        result.append(retriever.invoke(query))    
    print("==="* 20)
    return result

In [49]:
retrievers = {
    "bm25_faiss_73": bm25_faiss_73,
    "kiwibm25_faiss_37" : kiwibm25_faiss_37
}

temp = print_search_results(retrievers, "장애 유형은 몇 가지가 있나요?")

Query: 장애 유형은 몇 가지가 있나요?
bm25_faiss_73:  	 : 장애 유형은 15가지로 분류됩니다. 외부기능 장애 =
kiwibm25_faiss_37:  	 : 장애 유형은 15가지로 분류됩니다. 외부기능 장애 =


In [50]:
temp = print_search_results(retrievers, "장애 유형은 몇 가지가 있나요?")
print(temp[0]) # retriver bm25_faiss
print(temp[1]) # retriver kiwi(형태소) + bm25 + faiss

Query: 장애 유형은 몇 가지가 있나요?
bm25_faiss_73:  	 : 장애 유형은 15가지로 분류됩니다. 외부기능 장애 =
kiwibm25_faiss_37:  	 : 장애 유형은 15가지로 분류됩니다. 외부기능 장애 =
[Document(page_content='장애 유형은 15가지로 분류됩니다. 외부기능 장애 ='), Document(page_content='외부기능 장애 = [지체 장애, 청각 장애, 뇌병변'), Document(page_content='장애인 고용 대책 마련.'), Document(page_content='장애인 고용 전문가와 함께 다양한 장애인 고용 대책'), Document(page_content='장애인은 선천적인 경우보다 후천적인 경우가 훨씬'), Document(page_content='장애, 뇌병변 장애] 등'), Document(page_content='장애인 업무의 한계를 없애다')]
[Document(page_content='장애 유형은 15가지로 분류됩니다. 외부기능 장애 ='), Document(page_content='장애인은 선천적인 경우보다 후천적인 경우가 훨씬'), Document(page_content='장애, 뇌병변 장애] 등'), Document(page_content='외부기능 장애 = [지체 장애, 청각 장애, 뇌병변'), Document(page_content='장애인 업무의 한계를 없애다'), Document(page_content='경우가 훨씬 많습니다.')]


In [51]:
from langchain import LLMChain

template = """
다음 질문을 듣고 물음에 답해주세요.
{text}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

input_text = "장애 유형은 몇 가지가 있나요?"
# 4. LLMChain 만들기
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
result = llm_chain.invoke({"text": input_text})


질문에 답변드리자면, 장애는 신체적, 인지적, 정신적, 감각적 등 다양한 형태로 나타날 수 있습니다. 세계보건기구(WHO)에 따르면 전 세계적으로 약 10억 명의 사람들이 장애를 가지고 있으며, 이는 전 세계 인구의 약 15%에 해당합니다.

장애 유형은 다음과 같습니다:

* **신체적 장애:** 신체적 장애는 이동성 제한, 시력 또는 청력 손실, 만성 질환과 같은 신체 기능의 손상으로 인해 발생합니다.
* **인지적 장애:** 인지적 장애는 지적 능력이나 학습 능력의 결핍으로 인해 발생합니다. 예를 들어 자폐증, 다운 증후군, 주의력 결핍 과잉행동장애(ADHD) 등이 있습니다.
* **정신적 장애:** 정신적 장애는 정신 질환 또는 심리적 트라우마로 인해 발생합니다. 예를 들면 우울증, 불안 장애, 외상 후 스트레스 장애(PTSD) 등이 있습니다.
* **감각적 장애:** 감각적 장애는 청력 손실이나 시각 손상과 같은 감각 기능의 손상으로 인해 발생합니다.

장애 유형은 다양하며 각 개인에게 미치는 영향도 다릅니다. 어떤 사람들은 경미한 장애를 가지고 있어 일상 생활에 큰 영향을 받지 않을 수 있지만, 다른 이들은 중대한 장애를 가지고 있어 상당한 지원과 적응이 필요할 수 있습니다.

장애는 사람의 삶에 큰 영향을 미칠 수 있는 복잡한 문제입니다. 장애가 있는 사람들과 그들의 가족은 종종 도전과제에 직면하며, 이러한 도전을 극복하기 위해 자원과 지원이 필요합니다.

장애 유형을 이해하는 것은 장애가 있는 사람들을 이해하고 지원하는 데 있어 중요한 단계입니다. 장애의 다양한 형태를 인식함으로써 우리는 더 포괄적이고 접근 가능한 사회를 만들 수 있습니다.